In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
!pip install -q google-generativeai fpdf
!pip install -q google-adk

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━

In [10]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from IPython.display import display, clear_output, Markdown
import ipywidgets as widgets
import textwrap

instrucao_final = """
Voce é Reflexo um assistente virtual para momentos de dificuldades emocionais. Seja sucinto. expanda essa explicação e fale sobre suas potencialidades pelo contexto dos agentes.
Ao final da sua resposta, se fizer sentido, convide o usuário para um próximo passo, de forma emocional e gentil, sem termos técnicos.
Exemplos:
- “Se quiser, posso te ajudar a organizar isso com calma.”
- “Talvez agora seja uma boa hora de transformar esse sentimento em palavras.”
- “Se desejar, posso te ajudar a revisar o que foi escrito.”
- “Quer que eu procure algo que possa acolher esse momento?”
Nunca use palavras como agente, modelo ou pipeline.
"""

# Definição dos agentes
buscador = Agent(
    name="buscador",
    model="gemini-2.0-flash",
    instruction="""
Você é o Reflexo.Busca, um agente pesquisador emocional. Sua função é buscar, sugerir e interpretar conteúdos relevantes para a jornada de autoconhecimento do usuário.

Você deve:
- Sugerir leituras terapêuticas, autores, trechos, poesias e práticas introspectivas.
- Buscar abordagens psicológicas que se encaixem na dor ou na dúvida do usuário.
- Indicar recursos úteis de forma empática (sem parecer uma busca do Google).
- Sempre traduzir o que encontrou em linguagem acessível, emocional e segura.
""" + instrucao_final,
    tools=[google_search]
)

planejador = Agent(
    name="planejador",
    model="gemini-2.0-flash",
    instruction="Você é o Reflexo.Plano, um organizador emocional que cria rotinas e sugestões gentis para autocuidado." + instrucao_final,
)

redator = Agent(
    name="redator",
    model="gemini-2.5-pro-preview-03-25",
    instruction="Você é o Reflexo.Palavra, um redator da alma que transforma sentimentos em textos sinceros e libertadores." + instrucao_final,
)

revisor = Agent(
    name="revisor",
    model="gemini-2.5-pro-preview-03-25",
    instruction="Você é o Reflexo.Espelho, um revisor sensível que melhora textos sem apagar a emoção original." + instrucao_final,
)

agentes_disponiveis = {
    "Reflexo.Busca (pesquisar sugestões)": buscador,
    "Reflexo.Plano (organizar ideias)": planejador,
    "Reflexo.Palavra (escrever)": redator,
    "Reflexo.Espelho (revisar)": revisor
}

dropdown_agente = widgets.Dropdown(
    options=list(agentes_disponiveis.keys()),
    description='Agente:',
    layout=widgets.Layout(width='70%')
)

entrada = widgets.Text(placeholder='Digite sua mensagem aqui...', layout=widgets.Layout(width='100%'))
botao = widgets.Button(description="Enviar", button_style="primary")
saida = widgets.Output()

sessao_atual = None
runner = None

def iniciar_nova_sessao(nome_agente):
    global sessao_atual, runner
    agente = agentes_disponiveis[nome_agente]
    session_service = InMemorySessionService()
    sessao_atual = session_service.create_session(app_name=agente.name, user_id="user1", session_id="chat1")
    runner = Runner(agent=agente, app_name=agente.name, session_service=session_service)

def enviar_mensagem(b):
    user_input = entrada.value
    entrada.value = ''
    if not user_input.strip():
        return

    iniciar_nova_sessao(dropdown_agente.value)

    with saida:
        print(f"👤 Você: {user_input}")
        content = types.Content(role="user", parts=[types.Part(text=user_input)])
        final_response = ""
        for event in runner.run(user_id="user1", session_id="chat1", new_message=content):
            if event.is_final_response():
                for part in event.content.parts:
                    if part.text:
                        final_response += part.text + "\n"
        print(f"🧠 Reflexo: {final_response.strip()}\n")

botao.on_click(enviar_mensagem)

display(dropdown_agente, entrada, botao, saida)


Dropdown(description='Agente:', layout=Layout(width='70%'), options=('Reflexo.Busca (pesquisar sugestões)', 'R…

Text(value='', layout=Layout(width='100%'), placeholder='Digite sua mensagem aqui...')

Button(button_style='primary', description='Enviar', style=ButtonStyle())

Output()